# Creating a Table Data Source
We use BigQuery to load and prepare data for machine learning


---
## Setup

inputs:

In [96]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'group-24-361920'

In [97]:
REGION = 'us-west1'
EXPERIMENT = 'final_year_project_group24'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'Customer_Complaints'
BQ_TABLE = 'Customer_Complaints'

packages:

importing bigquery

In [98]:
from google.cloud import bigquery

clients:

creating a big query client for our project

In [99]:
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [100]:
BUCKET = PROJECT_ID + 'storage_bucket'

---
## Creating Dataset in BigQuery

Creating a dataset called Customer_Complaints in BigQuery

In [101]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

Showing that the dataset exists

In [102]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

Customer_Complaints


---
## Creating a Table in BigQuery
- We create a table asset in the BigQuery and populate it with the data from our dataset that we stored in the Cloud Storage Bucket


In [105]:

# if the table does not exist we create it and populate it with data from our dataset

from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        allow_quoted_newlines = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://group-24-361920storage_bucket/Customer_Complaints/dataset/Cusotmer_Complaints.csv", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: group-24-361920.Customer_Complaints.Customer_Complaints


In [106]:
# we can now directly query data from the created table in BigQuery
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
LIMIT 10
"""
bq.query(query = query).to_dataframe()


,int64_field_0,client_id,ticket_id,ticket_sent_to,ticket_type,fit_for_forum,fit_for_for_self_help,tier_level,contains_sensitive_data,ticket_text,has_media,ticket_source,ticket_created_at,first_reply_time_secs,response_id,response_text,response_source,response_created_at
0,33,1089318683242049536,1570173909982646276,MTN,Problem Ticket,False,False,TIER-0,False,@mtnug what is wrong with your data? This is n...,False,Twitter for iPhone,0022-09-14 22:13:00+00:00,267.000000,1570175030025416705,@AkechSonia please DM number and exact challen...,Khoros CX,9/14/22 10:17 PM
1,42,1107916375023079426,1570089690052567054,MTN,Problem Ticket,False,False,TIER-0,False,@mtnug I bought it yesterday but till now I ha...,False,Twitter for Android,0022-09-14 16:38:00+00:00,337.000000,1570091104455872517,"@kDavidTito Hello David, apologies, this has b...",Khoros CX,9/14/22 4:44 PM
2,66,748095549706997760,1569938088180289536,MTN,Problem Ticket,False,True,TIER-0,False,@mtnug How possible is it that my 2.9GB to be ...,False,Twitter Web App,0022-09-14 06:36:00+00:00,527.000000,1569940298360393729,"@mraycho_n Hello Racheal, sorry about that. Pl...",Khoros CX,9/14/22 6:45 AM
3,91,1403948459938484229,1569598944690454529,MTN,Problem Ticket,False,False,TIER-0,False,@mtnug the rate at which my data is moving yes...,False,Twitter for Android,0022-09-13 08:08:00+00:00,230.000000,1569599906393751553,"@NsimentaIsaac Hi Isaac, we'd love to look int...",Khoros CX,9/13/22 8:12 AM
4,170,1476158874507960325,1568451701098553344,MTN,Problem Ticket,False,False,TIER-0,False,Why is MTN extra time data too slow on Twitter...,False,Twitter for Android,0022-09-10 04:10:00+00:00,5457.000000,1568474590917148672,@DariusJackony @mtnug Y’ello! Thanks for stopp...,RC Engage Digital EU,9/10/22 5:40 AM
5,252,1174660738390069248,1567566766879948801,MTN,Problem Ticket,False,False,TIER-0,False,@mtnug your coverage is too reliable for your ...,False,Twitter for iPhone,0022-09-07 17:33:00+00:00,879.000001,1567570451701944320,"@Prnz_sbk Hello, aplogies, Kindly DM your numb...",Khoros CX,9/7/22 5:48 PM
6,303,2985946385,1566822249201541121,MTN,Problem Ticket,False,False,TIER-0,False,I have tried all means to replace my Simcards ...,False,Twitter for iPhone,0022-09-05 16:15:00+00:00,636.000000,1566824917512888322,"@GadhafiShafik Hello Gadhafi, Kindly DM your ...",Khoros CX,9/5/22 4:25 PM
7,313,1528391554741817345,1565776425478721536,MTN,Problem Ticket,True,False,TIER-0,False,"@mtnug my MTN agent line has failed to work, i...",False,Twitter for Android,0022-09-02 18:59:00+00:00,693.000000,1565779333632892929,"@Johnkamwada256 Hello John, kindly DM your num...",Khoros CX,9/2/22 7:10 PM
8,341,1487340852812959748,1566357451506696192,MTN,Problem Ticket,False,False,TIER-0,False,@mtnug should change on how it's consumes our ...,False,Twitter for Android,0022-09-04 09:28:00+00:00,847.000001,1566361004514099201,"@damas_muhumuza please DM your number, name, b...",Khoros CX,9/4/22 9:42 AM
9,350,1449717462610718722,1566285345146142720,MTN,Problem Ticket,False,True,TIER-0,False,@mtnug Hey @mtnug you refused to give me momo ...,False,Twitter for Android,0022-09-04 04:41:00+00:00,80.000000,1566285681168728064,"@Bravejonathano2 Hello, sorry about that. Plea...",Khoros CX,9/4/22 4:43 AM


---
## Reviewing Data with Pandas
We can now use additional SQL to review the data in BigQuery, we move data into a Pandas dataframe so that we can review it with python

In [107]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [109]:
df['ticket_type'].value_counts()

Problem Ticket           408
Service Request          271
Unclassified Ticket      149
Incident Ticket          107
Change Request Ticket     68
Name: ticket_type, dtype: int64

In [110]:
df['fit_for_forum'].value_counts()

False    534
True     469
Name: fit_for_forum, dtype: int64

In [113]:
df['fit_for_for_self_help'].value_counts()

False    812
True     191
Name: fit_for_for_self_help, dtype: int64

In [112]:
df['tier_level'].value_counts()

TIER-1    563
TIER-0    351
TIER-2     73
TIER-3     13
TIER-4      3
Name: tier_level, dtype: int64

---
## Preparing Data for Machine Learning

We split the data into three categories, Training data, Testing data and Vadidating data with percentages.
We create a copy of the table in BigQuery and add a splits column to it that defines the category of each row.

- Training Data: **82.552343%** 
- Testing Data: **7.976072%**
- Validating Data: **9.471585%**

In [114]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_copy` AS
WITH add_id AS(SELECT *, GENERATE_UUID() row_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(row_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(row_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

We can now review our splitted data

In [115]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_copy`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,80,7.976072
1,VALIDATE,95,9.471585
2,TRAIN,828,82.552343


Retrieve a subset of the data to a Pandas dataframe:

In [116]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_copy`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [117]:
data.head()

,int64_field_0,client_id,ticket_id,ticket_sent_to,ticket_type,fit_for_forum,fit_for_for_self_help,tier_level,contains_sensitive_data,ticket_text,has_media,ticket_source,ticket_created_at,first_reply_time_secs,response_id,response_text,response_source,response_created_at,row_id,splits
0,232,224961061,1567776119495053312,MTN,Problem Ticket,False,False,TIER-1,False,"@mtnug what’s happening?, can’t access my MoKa...",False,Twitter for iPhone,0022-09-08 07:25:00+00:00,551.0,1567778431462408192,@alvinagume that is unfortunate. DM your numbe...,Khoros CX,9/8/22 7:34 AM,8f172c25-847e-4f95-9d38-56fbf5715e3b,TEST
1,434,1386995671031586817,1565358180753817602,MTN,Problem Ticket,False,False,TIER-2,False,@mtnug I can't make a withdrawal 😭,False,Twitter for Android,0022-09-01 15:17:00+00:00,541.0,1565360449377402880,"@soon_quiting Hello, sorry about that, please ...",Khoros CX,9/1/22 3:26 PM,d50b1426-aed8-46db-9deb-87300e04e09c,VALIDATE
2,40,1022961189985419269,1570100143331987458,MTN,Problem Ticket,True,False,TIER-2,False,@mtnug @najjits Your internet ain’t working …....,False,Twitter for iPhone,0022-09-14 17:20:00+00:00,850.0,1570103707932708867,"@HermanNamanya Hello Herman, apologies, kindly...",Khoros CX,9/14/22 5:34 PM,238379f4-9f51-4dfd-aa33-85aad03a7f50,TRAIN
3,110,2383425320,1569347320591835137,MTN,Problem Ticket,True,False,TIER-3,False,@ojok_ayen @CentenaryBank In 2020 my phone was...,False,Twitter for Android,0022-09-12 15:28:00+00:00,695.0,1569350234567843841,@mathiasowilli We are sorry to hear this but c...,Khoros CX,9/12/22 3:40 PM,a3e2ee77-77f9-457e-b75a-cb4a976167ad,VALIDATE
4,407,282839615,1565720189114748930,MTN,Problem Ticket,False,False,TIER-0,False,"@Airtel_Ug , why is my line not connecting wit...",False,Twitter Web App,0022-09-02 15:15:00+00:00,194.0,1565721004202229760,"@samgidudu Hello Samuel , kindly DM your numbe...",Khoros CX,9/2/22 3:19 PM,20214f3b-38e5-46f9-bbfa-09940780c06d,TRAIN
